1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [1]:
# Importing libraries
import pandas as pd
import selenium
import time
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import requests
import re

In [2]:
# Importing warings and ignoring waring if any present
import warnings
warnings.filterwarnings('ignore')

In [27]:
# connect to webdriver
driver = webdriver.Chrome("chromedriver.exe")

# get webpage
driver.get("https://www.amazon.in/")
driver.maximize_window()
time.sleep(2)

In [4]:
# search the user input product
search_bar = driver.find_element_by_id("twotabsearchtextbox")    
search_bar.clear()                                               
search_bar.send_keys(input('Enter the product you want to search : '))
time.sleep(3)
driver.find_element_by_xpath('//div[@class="nav-search-submit nav-sprite"]/span/input').click() 

Enter the product you want to search : laptop


# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [85]:
from selenium.webdriver.common.by import By

In [7]:
b_name=[]#Brand name
product_name=[]#Product Name
price=[]#Price
ret_exchange=[]#return/Exchange
exp_delivery=[]#Expected delivery
availability=[]#Availability
product_url=[]
urls=[]
for i in range(0,3):
    if i==0:
        p_urls=driver.find_elements(By.XPATH,"//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a")
    if i==1:
        n_button=driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div[1]/div/span[3]/div[2]/div[29]/div/div/span/a[1]')
        n_button.click()
        time.sleep(2)
        p_urls=driver.find_elements(By.XPATH,"//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a")
    # selecting and clicking next page
    if i==2:
        n_button=driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div[1]/div/span[3]/div[2]/div[28]/div/div/span/a[3]')
        n_button.click()
        time.sleep(2)
        p_urls=driver.find_elements(By.XPATH,"//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a")
            
            

    for i in p_urls:
        urls.append(i.get_attribute('href'))

    
for i in urls:
    driver.get(i)
    time.sleep(2)
#scraping data for brand name
    try:
        bran=driver.find_element(By.XPATH,"//a[@id='bylineInfo']")
        b_name.append(bran.text)
    except NoSuchElementException as e:
        b_name.append("--")

#scraping data for product name
    try:
        prod=driver.find_element(By.XPATH,"//h1[@id='title']/span")
        product_name.append(prod.text)
    except NoSuchElementException as e:
        product_name.append("--")


#scraping data for price
    try:
        pri=driver.find_element(By.XPATH,"//div[@id='apex_desktop']/div/div/table/tbody/tr[2]/td[2]/span/span[2]")
        price.append(pri.text)
    except NoSuchElementException as e:
            price.append("--")

#scaping data for return
    try:
        retu=driver.find_element(By.XPATH,"//div[@id='RETURNS_POLICY']/span/div[2]/a")
        ret_exchange.append(retu.text)
    except NoSuchElementException as e:
        ret_exchange.append("--")

#scraping data for expected delivery
    try:
        deliv=driver.find_element(By.XPATH,"//div[@id='ddmDeliveryMessage']/b")
        exp_delivery.append(deliv.text)
    except NoSuchElementException as e:
        exp_delivery.append("--")

#scraping data for availability
    try:
        avail=driver.find_element(By.XPATH,"//div[@id='availability']/span")
        availability.append(avail.text)
    except NoSuchElementException as e:
        availability.append("--")
              
            
     
    

In [9]:
print(len(availability),len(b_name),len(product_name),len(ret_exchange),len(exp_delivery),len(availability),len(price),len(urls))

70 70 70 70 70 70 70 70


In [10]:
#Placing everything into dataframe
df=pd.DataFrame({"Brand Name":b_name,"Product":product_name,"Price":price,"Return/Exchange":ret_exchange,"Expected Delivery":exp_delivery,"Availability":availability})#"Product Url":urls})
df

,Brand Name,Product,Price,Return/Exchange,Expected Delivery,Availability
0,Visit the Lenovo Store,Lenovo IdeaPad 1 Intel Celeron N4020 11.6'' HD...,--,7 Days Replacement,--,In stock.
1,Visit the HP Store,HP 15s- Ryzen 5- 8GB RAM/512GB SSD 15.6 Inches...,--,7 Days Replacement,--,In stock.
2,Visit the ASUS Store,"ASUS VivoBook 15 (2021), 15.6-inch (39.62 cm) ...",--,7 Days Replacement,--,In stock.
3,Visit the Dell Store,"Dell New Inspiron 3525 Laptop, AMD Athlon Silv...",--,7 Days Replacement,--,In stock.
4,Visit the Lenovo Store,Lenovo IdeaPad 1 Intel Celeron N4020 11.6'' HD...,--,7 Days Replacement,--,In stock.
...,...,...,...,...,...,...
65,Visit the HP Store,Hp 14S-Intel Pentium Silver N6000- 8Gb Ram/256...,--,7 Days Replacement,--,In stock.
66,Visit the Dell Store,"Dell Vostro 3400 Laptop, Intel I3-1115G4, 4Gb,...",--,7 Days Replacement,--,In stock.
67,Visit the Dell Store,"Dell Inspiron 3515 Laptop, AMD Ryzen 3-3250U 1...",--,7 Days Replacement,--,In stock.
68,Visit the ASUS Store,"ASUS TUF Gaming F15 (2021), 15.6-inch (39.62 c...",--,7 Days Replacement,--,Only 1 left in stock.


In [12]:
df.to_csv("Scraped Data",index=False)

In [13]:
driver.close()

#  3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [25]:

a = ['fruits','cars','Machine Learning','Guitar','Cakes']
for k in a:
    driver = webdriver.Chrome("chromedriver.exe") # connect to webdriver
    driver.get('https://www.google.co.in/imghp?hl=en&tab=ri&ogbl')
       # get webpage
    search = driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
    search.send_keys(k)
    time.sleep(3)
    
    searchbtn = driver.find_element_by_xpath('//button[@class = "Tg7LZd"]') # click search button
    searchbtn.click()
    time.sleep(5)
    imgsrc = []
    imgurl = []
   
    i=0
    while(i<30):
        driver.execute_script("window.scrollBy(0,50000)") # scroll down to get more images
        i+=1
    time.sleep(2)
    image_tag = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")
    for i in image_tag:
        imgurl.append(i.get_attribute('src'))
    for i in imgurl:
        if i != None and i[0:8] =='https://': #extracing image urls
             imgsrc.append(i)
        
    for i in range(0,10):
        d = requests.get(imgsrc[i])
        with open("C:/Users\RAVISANKAR P K/Documents/fliprobo"+k+str(i)+".png","wb") as file: #opening as file to be written in binary mode
        
          file.write(d.content) #saving image in direcotry from source url
    driver.quit()

In [28]:
driver.close()

# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) onwww.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [54]:
# connect to webdriver
driver = webdriver.Chrome("chromedriver.exe")

# get webpage
driver.get("https://www.flipkart.com/")
time.sleep(2)
driver.maximize_window()

In [55]:
driver.find_element_by_xpath('/html/body/div[2]/div/div/button').click()

In [56]:
# search the user input product

search_bar = driver.find_element_by_xpath('//input[@class="_3704LK"]') 
search_bar.clear()                                               
search_bar.send_keys(input('Enter the name of a smartphone to search : '))  
time.sleep(2)
search_button = driver.find_element_by_xpath('//button[@class="L0Z3Pu"]') 
search_button.click() 

Enter the name of a smartphone to search : poco


In [57]:
produrl = []

produrl_tag = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")#getting product url
for u in produrl_tag:
     produrl.append(u.get_attribute("href"))

In [58]:
len(produrl)

24

In [59]:
Brand = []
prod = []
Colour = []
Ram = []
Rom = []
PrimCamera = []
SecCamera = []
DispSize = []
Price = []
Prourl = []
BatteryCap = []


for i in produrl:
    driver.get(i)
    time.sleep(3)
    morebtn = driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']")
    morebtn.click()
    time.sleep(3)
    #scraping data for brand
    try:
        brandtag = driver.find_element_by_xpath("/html/body/div[1]/div/div[3]/div[1]/div[2]/div[1]/div[1]/div/div[4]/a")
        Brand.append(brandtag.text)
        Brand = Brand[0:24]
    except NoSuchElementException as e:
        Brand.append('-')
        
    #scraping data for product name   
    try:
        prodtag = driver.find_element_by_xpath("//span[@class='B_NuCI']")
        prod.append(prodtag.text)
        prod = prod[0:24]
    except NoSuchElementException as e:
        prod.append('-')
        
    #scraping data for colour    
    try:
        Colourtag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][1]/table/tbody/tr[4]/td[2]/ul/li")
        Colour.append(Colourtag.text)
        Colour = Colour[0:24]
    except NoSuchElementException as e:
        Colour.append('-')
        
    #scraping data for Ram    
    try:
        RAMtag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][4]/table/tbody/tr[2]/td[2]/ul/li")
        Ram.append(RAMtag.text)
        Ram = Ram[0:24]
    except NoSuchElementException as e:
        RAM.append('-')
        
    #scraping data for price    
    try:
        pricetag = driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']")
        price.append(pricetag.text)
        price = price[0:24]
    except NoSuchElementException as e:
        price.append('-')
    
    #scraping data for storage
    try:
        Storagetag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][4]/table/tbody/tr[1]/td[2]/ul/li")
        Rom.append(Storagetag.text)
        Rom = Rom[0:24]
    except NoSuchElementException as e:
        Storage.append('-')
    
    #scraping data for primary camera
    try:
        PrimCameratag = driver.find_element_by_xpath("//div[@class='_2418kt']/ul/li[3]")
        PrimCamera.append(PrimCameratag.text.split('|', 1)[0])
        PrimCamera = PrimCamera[0:24]
    except NoSuchElementException as e:
        PrimCamera.append('-')
        
    #scraping data for sec camera
    try:
        SecCameratag = driver.find_element_by_xpath("//div[@class='_2418kt']/ul/li[3]")
        SecCamera.append(SecCameratag.text.split('|', 1)[0])
        SecCamera = SecCamera[0:24]
    except NoSuchElementException as e:
        SecCamera.append('-')
        
    #scraping data for display
    try:
        DispSizetag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][2]/table/tbody/tr[1]/td[2]/ul/li")
        DispSize.append(DispSizetag.text)
        DispSize = DispSize[0:24]
    except NoSuchElementException as e:
        DispSize.append('-')
        
    
        
     #scraping data for battery               
    try:
        BatteryCap_tag = driver.find_element_by_xpath("//div[@class='_2418kt']/ul/li[4]")
        BatteryCap.append(BatteryCap_tag.text)
        BatteryCap = BatteryCap[0:24]
    except NoSuchElementException as e:
        BatteryCap.append('-') 
    
    
    

In [62]:
len(Brand),len(prod),len(price),len(Colour),len(Ram),len(Rom),len(PrimCamera),len(SecCamera),len(DispSize),len(BatteryCap)


(24, 24, 24, 24, 24, 24, 24, 24, 24, 24)

In [65]:
smartphonedf  = pd.DataFrame({'Brand Name':Brand,'Smartphone Name':prod,'Price(₹)':price,'Colour':Colour,'RAM':Ram ,'Storage(ROM)':Rom,'Primary Camera':PrimCamera,'Secondary Camera':SecCamera,'Display Size':DispSize,'Battery Capacity':BatteryCap,'Product URL':produrl})

smartphonedf

,Brand Name,Smartphone Name,Price(₹),Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Product URL
0,POCO Mobiles,"POCO C31 (Royal Blue, 64 GB) (4 GB RAM)",--,Royal Blue,4 GB,64 GB,13MP + 2MP + 2MP,13MP + 2MP + 2MP,16.59 cm (6.53 inch),5000 mAh Lithium-ion Polymer Battery,https://www.flipkart.com/poco-c31-royal-blue-6...
1,POCO Mobiles,"POCO C31 (Shadow Gray, 64 GB) (4 GB RAM)",--,Shadow Gray,4 GB,64 GB,13MP + 2MP + 2MP,13MP + 2MP + 2MP,16.59 cm (6.53 inch),5000 mAh Lithium-ion Polymer Battery,https://www.flipkart.com/poco-c31-shadow-gray-...
2,POCO Mobiles,"POCO C31 (Royal Blue, 32 GB) (3 GB RAM)",--,Royal Blue,3 GB,32 GB,13MP + 2MP + 2MP,13MP + 2MP + 2MP,16.59 cm (6.53 inch),5000 mAh Lithium-ion Polymer Battery,https://www.flipkart.com/poco-c31-royal-blue-3...
3,POCO Mobiles,"POCO C31 (Shadow Gray, 32 GB) (3 GB RAM)",--,Shadow Gray,3 GB,32 GB,13MP + 2MP + 2MP,13MP + 2MP + 2MP,16.59 cm (6.53 inch),5000 mAh Lithium-ion Polymer Battery,https://www.flipkart.com/poco-c31-shadow-gray-...
4,POCO Mobiles,"POCO M4 Pro (Power Black, 64 GB) (6 GB RAM)",--,Power Black,6 GB,64 GB,64MP + 8MP + 2MP,64MP + 8MP + 2MP,16.33 cm (6.43 inch),5000 mAh Lithium-ion Polymer Battery,https://www.flipkart.com/poco-m4-pro-power-bla...
5,POCO Mobiles,"POCO M4 Pro (Cool Blue, 128 GB) (6 GB RAM)",--,Cool Blue,6 GB,128 GB,64MP + 8MP + 2MP,64MP + 8MP + 2MP,16.33 cm (6.43 inch),5000 mAh Lithium-ion Polymer Battery,https://www.flipkart.com/poco-m4-pro-cool-blue...
6,POCO Mobiles,"POCO M4 Pro (Power Black, 128 GB) (6 GB RAM)",--,Power Black,6 GB,128 GB,64MP + 8MP + 2MP,64MP + 8MP + 2MP,16.33 cm (6.43 inch),5000 mAh Lithium-ion Polymer Battery,https://www.flipkart.com/poco-m4-pro-power-bla...
7,POCO Mobiles,"POCO M4 Pro 5G (Cool Blue, 128 GB) (6 GB RAM)",--,Cool Blue,6 GB,128 GB,50MP + 8MP,50MP + 8MP,16.76 cm (6.6 inch),5000 mAh Lithium-ion Polymer Battery,https://www.flipkart.com/poco-m4-pro-5g-cool-b...
8,POCO Mobiles,"POCO M4 Pro (Cool Blue, 64 GB) (6 GB RAM)",--,Cool Blue,6 GB,64 GB,64MP + 8MP + 2MP,64MP + 8MP + 2MP,16.33 cm (6.43 inch),5000 mAh Lithium-ion Polymer Battery,https://www.flipkart.com/poco-m4-pro-cool-blue...
9,POCO Mobiles,"POCO M4 5G (Cool Blue, 64 GB) (4 GB RAM)",--,Cool Blue,4 GB,64 GB,50MP + 2MP,50MP + 2MP,16.71 cm (6.58 inch),5000 mAh Lithium Ion Polymer Battery,https://www.flipkart.com/poco-m4-5g-cool-blue-...


In [66]:
smartphonedf.to_csv("Smartphone")

In [67]:
driver.close()

# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps

In [68]:
# connect to webdriver
driver = webdriver.Chrome("chromedriver.exe")

# get webpage
driver.get("https://www.google.com/maps")
driver.maximize_window()
time.sleep(2)

In [71]:
srch_bar = driver.find_element(By.XPATH,'//input[@class="tactile-searchbox-input"]')    
srch_bar.clear()                                               
srch_bar.send_keys(input('Enter the city name: ')) 
#finding the search button and clicking
srch_button = driver.find_element(By.XPATH,'//button[@id="searchbox-searchbutton"]')       
srch_button.click()
time.sleep(3)

Location=driver.current_url
try:
    coordinates=re.findall(r'@(.*)data',Location)
    lat_lon=coordinates[0].split(",")
    lat=lat_lon[0]
    lon=lat_lon[1]
except IndexError as e:
    coordinates=re.findall(r'@(.*)z',Location)
    lat_lon=coordinates[0].split(",")
    lat=lat_lon[0]
    lon=lat_lon[1]
print("\nlatitude={},longitude={}".format(lat,lon))

Enter the city name: mumbai

latitude=19.0821978,longitude=72.7411017


In [72]:
driver.close()

# 6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in

In [9]:
driver = webdriver.Chrome("chromedriver.exe")

In [10]:
# get webpage
driver.get("https://trak.in/")
time.sleep(2)
driver.maximize_window()


In [11]:
#click on funding deals and go to that page
fun_btn=driver.find_element_by_xpath("//li[@id='menu-item-51510']/a").get_attribute('href')
driver.get(fun_btn)

In [22]:
# Creating empty list
funding={}
funding["Date"]=[]
funding["Startup Name"]=[]
funding["Industry or Vertical"]=[]
funding["Sub Vertical"]=[]
funding["Location"]=[]
funding["Investor"]=[]
funding["Investment Type"]=[]
funding["Amount"]=[]

for i in range(54,57):


    # Scraping data of Date
    date_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))# {} and .format () used
    #so the number coming in function format(n) will replace {}
    for date in date_tags:
        funding['Date'].append(date.text)

    # Scraping data of Startup Name
    name_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for name in name_tags:
        funding['Startup Name'].append(name.text)
    
    # Scraping data of Industry OR Vertical
    ind_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind_tags:
        funding['Industry or Vertical'].append(n.text)
    
    # Scraping data of Sub-Vertical
    sv_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for sv in sv_tags:
        funding['Sub Vertical'].append(sv.text)

    # Scraping data of Location
    loc_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for loc in loc_tags:
        funding['Location'].append(loc.text)
    
    # Scraping data of Investor
    inv_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for inv in inv_tags:
        funding['Investor'].append(inv.text)
        
    # Scraping data of Investment Type
    invt_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for invt in invt_tags:
        funding['Investment Type'].append(invt.text)
    # Scraping data of Amount
    amt_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for amt in amt_tags:
        funding['Amount'].append(amt.text)

In [23]:
details=pd.DataFrame(funding)
details

,Date,Startup Name,Industry or Vertical,Sub Vertical,Location,Investor,Investment Type,Amount
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


In [24]:
driver.close()

# 7. Write a program to scrap all the available details of best gaming laptops from digit.in

In [27]:
driver=webdriver.Chrome("chromedriver.exe")
driver.maximize_window()

In [28]:
driver.get("https://www.digit.in/")

In [29]:
driver.find_element_by_xpath("//div[@class='listing_container']//ul//li[9]").click()
time.sleep(3)

In [46]:
#Creating empty lists
Laptop_Name = []
Operating_sys = []
Display = []
Processor = []
Memory = []
Weight = []
Dimensions = []
Graph_proc = []
Price = []
# Scraping the data of laptop names
laptop_name= driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for name in laptop_name:
    Laptop_Name.append(name.text)
    
    
#Scraping the data of operating system
try:
    op_sys = driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")
    for os in op_sys:
        Operating_sys.append(os.text)
except NoSuchElementException:
    pass


#Scraping data of display of the laptop
try:
    display= driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")
    for disp in display:
        Display.append(disp.text)
except NoSuchElementException:
    pass


#Scraping data of Processor
try:
    processor = driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[5]/td[3]")
    for pro in processor:
        Processor.append(pro.text)
except NoSuchElementException:
    pass

#Scraping data of memory
try:
    memory = driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")
    for memo in memory:
        Memory.append(memo.text)
except NoSuchElementException:
    pass



#Scraping data of dimensions
try:
    dimension = driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[8]/td[3]")
    for dim in dimension:
        Dimensions.append(dim.text)
except NoSuchElementException:
    pass


#Scraping data of Graph processor
try:
    graph = driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[7]/td[3]")
    for gra in graph:
        Graph_proc.append(gra.text)
except NoSuchElementException:
    pass


#Scraping data of price
try:
    price = driver.find_elements_by_xpath("//td[@class='smprice']")
    for pri in price:
        Price.append(pri.text.replace('₹','Rs '))
except NoSuchElementException:
    pass

In [47]:
print(len(Laptop_Name),len(Operating_sys),len(Display),len(Processor),len(Memory),len(Dimensions),len(Graph_proc),len(Price))


10 10 10 10 10 10 10 10


In [48]:
Gaming_Laptop=pd.DataFrame({})
Gaming_Laptop['Laptop Name'] = Laptop_Name
Gaming_Laptop['Operating system'] = Operating_sys
Gaming_Laptop['Display'] = Display
Gaming_Laptop['Processor'] = Processor
Gaming_Laptop['Memory'] = Memory
Gaming_Laptop['Dimensions'] = Dimensions
Gaming_Laptop['Graphical Processor'] = Graph_proc
Gaming_Laptop['Price'] = Price
Gaming_Laptop

,Laptop Name,Operating system,Display,Processor,Memory,Dimensions,Graphical Processor,Price
0,MSI RAIDER GE76,WINDOWS 11 HOME OS,"17"" (3840 X 2160) DISPLAY",Windows 11 Home,16 GB DDR5 RAM & 2 TB SSD,397 x 284 x 26 mm dimension & 2.9 kg weight,NVIDIA GeForce RTX 3080Ti Graphics card,"Rs 429,940"
1,ASUS ROG STRIX SCAR 15,WINDOWS 11 HOME OS,"15.6"" (2560 X 1440) DISPLAY",Windows 11 Home,32 GB DDR5 RAM & 2 TB SSD,259 x 354 x 27 mm dimension & 2.3 kg weight,8 GB DDR6 NVIDIA GeForce RTX 3070 Ti Graphics ...,"Rs 280,990"
2,ACER NITRO 5,WINDOWS 10 OS,"15.6"" (1920 X 1080) DISPLAY",Windows 10,16 GB DDR4 RAM & 1 TB HDD with 256 GB SSD,363.4 x 255 x 23.9 mm dimension & 2.4 kg weight,NVIDIA GeForce RTX 3070 Graphics card,"Rs 129,990"
3,MSI STEALTH 15M,WINDOWS 10 OS,"15.6"" (1920 X 1080) DISPLAY",Windows 10,16 GB DDR4 RAM & 1 TB SSD,358.3 x 248 x 16.15 mm dimension & 1.7 kg weight,NVIDIA GeForce RTX 3060 Graphics card,"Rs 134,990"
4,ASUS ROG STRIX SCAR 15,WINDOWS 10 OS,"15.6"" (2560 X 1440) DISPLAY",Windows 10,32 GB DDR4 RAM & 2 TB SSD,354 x 259 x 22.6 mm dimension & 2.30 kg weight,NVIDIA GeForce RTX 3080 Graphics card,"Rs 193,990"
5,ASUS ROG STRIX SCAR 15,WINDOWS 10 HOME OS,"15.6"" (1920 X 1080) DISPLAY",Windows 10 Home,16 GB DDR4 RAM & 1 TB SSD,35.4 x 25.9 x 2.26 mm dimension & 2.30 kg weight,NVIDIA® GeForce RTX™ 3070 Graphics card,N/A
6,ASUS ZEPHYRUS G14,WINDOWS 10 HOME OS,"14"" (1920 X 1080) DISPLAY",Windows 10 Home,16 GB DDR4 RAM & 1 TB SSD,32.5 x 22.1 x 1.8 mm dimension & 1.65 kg weight,NVIDIA GeForce RTX 2060 Graphics card,"Rs 144,990"
7,HP OMEN 16,WINDOWS 11 HOME OS,"16.1"" (1920 X 1080) DISPLAY",Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,36.92 x 24.8 x 2.3 mm dimension & 2.32 kg weight,6 GB GDDR6 NVIDIA GeForce RTX 3060 Graphics card,"Rs 145,500"
8,ASUS ROG ZEPHYRUS DUO 15,WINDOWS 10 OS,"15.6"" (3840 X 1100) DISPLAY",Windows 10,4 GB DDR4 RAM & 512 GB SSD,268.30 x 360.00 x 20.90 mm dimension & 2.4 kg ...,NVIDIA GeForce RTX 2070 Max-Q Graphics card,"Rs 185,000"
9,ACER ASPIRE 7 GAMING LAPTOP,WINDOWS 10 HOME OS,"15.6"" (1920 X 1080) DISPLAY",Windows 10 Home,8 GB DDR4 RAM & 512 GB SSD,2.29 x 36.3 x 25.4 mm dimension & 2.15 kg weight,NVIDIA® GeForce® GTX 1650 Graphics card,"Rs 53,490"


In [58]:
driver.close()

# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”

In [87]:
driver=webdriver.Chrome("chromedriver.exe")
driver.maximize_window()

In [88]:

# get webpage
driver.get("https://www.forbes.com/?sh=63b020902254")
time.sleep(2)

In [89]:
 #clicking on Explore button
Explbtn = driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[1]/button[1]')
Explbtn.click()
time.sleep(3)

In [90]:
#select billionaire  
driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]").click()
time.sleep(2)

In [91]:
#select world billionaire  
world_bill= driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")
world_bill.click()
time.sleep(1)

In [92]:
#Creating empty lists
Rank = []
Person_Name = []
total_net_worth = []
Age = []
citizenship = []
Source = []
industry = []

while(True):
    #Scraping the data of rank of the billionaires
    rank_tags= driver.find_elements_by_xpath("//div[@class='rank']")
    for rank in rank_tags:
        Rank.append(rank.text)
    
    
    #Scraping the data of names of the billionaires
    name_tags= driver.find_elements_by_xpath("//div[@class='personName']/div")
    for name in name_tags:
        Person_Name.append(name.text)
    
    
    #Scraping data of age of the billionaires
    age_tags= driver.find_elements_by_xpath("//div[@class='age']/div")
    for age in age_tags:
        Age.append(age.text)   
    
    
    #Scraping data of citizenship of the billionaires
    cit_tags= driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for cit in cit_tags:
        citizenship.append(cit.text)
    
    
    #Scraping data of source of income of the billionaires
    sour_tags= driver.find_elements_by_xpath("//div[@class='source']")
    for sour in sour_tags:
        Source.append(sour.text)    

    
    
    #Scraping data of Industry of the billionaires
    ind_tags= driver.find_elements_by_xpath("//div[@class='category']//div")
    for ind in ind_tags:
        industry.append(ind.text)
        
        
    #scraping data of net_worth of billionaires
    net_tags= driver.find_elements_by_xpath("//div[@class='netWorth']//div[1]")
    for net in net_tags:
        total_net_worth.append(net.text.split(',')[0])
    
    
    #Clicking on next button
    try:
        next_button = driver.find_element_by_xpath("//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click()
        time.sleep(2)
    except:
        break
total_net_worth=total_net_worth[0::2]

In [93]:
print(len(Rank),len(Person_Name),len(Age),len(citizenship),len(Source),len(industry),len(total_net_worth))

2668 2668 2668 2668 2668 2668 2668


In [95]:
#Framing data
Billionaires=pd.DataFrame({})
Billionaires['Rank'] = Rank
Billionaires['Names'] = Person_Name
Billionaires['Net Worth'] = total_net_worth
Billionaires['Age'] = Age
Billionaires['Citizenship'] = citizenship
Billionaires['Source'] = Source
Billionaires['Industry'] = industry
Billionaires

,Rank,Names,Net Worth,Age,Citizenship,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
2663,2578.,Zhang Yuqiang,$1 B,66,China,Fiberglass,Manufacturing
2664,2578.,Zhou Ruxin,$1 B,59,China,Navigation,Technology
2665,2578.,Wen Zhou & family,$1 B,57,China,chemicals,Manufacturing
2666,2578.,Zhou Yifeng & family,$1 B,43,China,liquefied petroleum gas,Energy


In [96]:
driver.close()

# 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video

In [124]:
driver=webdriver.Chrome('chromedriver.exe')
driver.maximize_window()

In [125]:
# Opening the youtube.com
url = "https://www.youtube.com/"
driver.get(url)
time.sleep(2)

In [126]:
search_bar = driver.find_element_by_xpath("/html/body/ytd-app/div[1]/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/form/div[1]/div[1]/input")
search_bar
search_bar.send_keys("Money Heist")

In [127]:
search_btn = driver.find_element_by_id("search-icon-legacy")  
search_btn.click()
time.sleep(1)

In [128]:
#Clicking on first video
video = driver.find_element_by_xpath("//yt-formatted-string[@class ='style-scope ytd-video-renderer']")
video.click()

In [129]:
# 1000 time we scroll down by 10000 in order to generate more Comments
for _ in range(20000):
    driver.execute_script("window.scrollBy(0,20000)")
time.sleep(2)

In [162]:
#Creating empty lists
comments = []
comment_time = []
Time = []
Likes = []
No_of_Likes = []
#scrape comments
cm = driver.find_elements_by_id("content-text")
for i in cm:
    if (len(comments)<500):
        if i.text is None:
            comments.append("--")
        else:
            comments.append(i.text)
time.sleep(2)
# scrape time when comment was posted
tm = driver.find_elements_by_xpath("//a[contains(text(),'ago')]")
for i in tm:
    if (len(Time)<500):
        Time.append(i.text)

for i in range(0,len(Time)):
    comment_time.append(Time[i])
time.sleep(5)
    
# scrape the comment likes
like = driver.find_elements_by_xpath('//span[@class="style-scope ytd-comment-action-buttons-renderer"]')
for i in like:
    if (len(Likes)<1000):
        if i.text is None:
            Likes.append("--")
        else:
        
            Likes.append(i.text)
Likes=Likes[1::2]    
#for i in range(1,len(Likes)):
    #No_of_Likes.append(Likes[i])

In [163]:
print(len(comments),len(comment_time),len(Likes))

500 500 500


In [164]:
#Likes

['2.9K',
 '4.5K',
 '940',
 '316',
 '397',
 '209',
 '2.2K',
 '204',
 '112',
 '185',
 '70',
 '9',
 '21',
 '10',
 '14',
 '30',
 '11',
 '872',
 '6',
 '6',
 '948',
 '9',
 '12',
 '4',
 '15',
 '1',
 '229',
 '2',
 '7',
 '6',
 '',
 '6',
 '305',
 '6',
 '9',
 '29',
 '1',
 '4',
 '183',
 '2',
 '372',
 '13',
 '32',
 '',
 '',
 '10',
 '77',
 '3',
 '1',
 '2',
 '2',
 '1',
 '110',
 '1',
 '',
 '2',
 '1',
 '2',
 '153',
 '',
 '90',
 '1',
 '2',
 '',
 '',
 '',
 '53',
 '',
 '',
 '3',
 '1',
 '2',
 '45',
 '2',
 '',
 '9',
 '2',
 '',
 '272',
 '',
 '19',
 '1',
 '1',
 '1',
 '2',
 '',
 '12',
 '2',
 '',
 '',
 '',
 '',
 '40',
 '',
 '',
 '1',
 '1',
 '6',
 '24',
 '',
 '67',
 '17',
 '',
 '1',
 '1',
 '1',
 '6',
 '2',
 '',
 '1',
 '1',
 '3',
 '7',
 '12',
 '',
 '',
 '',
 '3',
 '4',
 '',
 '3',
 '1',
 '',
 '1',
 '',
 '',
 '18',
 '',
 '1',
 '',
 '1',
 '',
 '5',
 '',
 '',
 '2',
 '',
 '',
 '4',
 '',
 '15',
 '',
 '3',
 '1',
 '1',
 '2',
 '10',
 '5',
 '1',
 '3',
 '',
 '9',
 '10',
 '',
 '1',
 '2',
 '',
 '1',
 '6',
 '2',
 '15',
 '1',
 

In [165]:
# dataframe for scraped data

Youtube=pd.DataFrame({})
Youtube['Comments'] = comments
Youtube['Comment_time'] = comment_time
Youtube['Comment upvotes']=Likes
Youtube

,Comments,Comment_time,Comment upvotes
0,Money Heist is definitely one of the shows I h...,6 months ago,2.9K
1,"This show truly is a masterpiece, sad it’s com...",6 months ago,4.5K
2,What a masterpiece. I just have finished to wa...,6 months ago (edited),940
3,We need more series like this one....this seri...,6 months ago,316
4,Can't explain the emotions this trailer evoked...,6 months ago,397
...,...,...,...
495,No one get out of the bank of Spain alive but ...,6 months ago,
496,Best show ending ever,6 months ago,
497,"Gracias casa de papel for the serie ,,its my f...",6 months ago,
498,Money Heist Fans: Watch each season as soon ...,6 months ago,


In [166]:
driver.close()

# Q10: Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [43]:
driver=webdriver.Chrome("chromedriver.exe")
driver.maximize_window()
time.sleep(2)
# Getting the web page of mentioned url
url = "https://www.hostelworld.com/"
driver.get(url)
time.sleep(5)

In [44]:
#select london
driver.find_element_by_id('location-text-input-field').click()
time.sleep(2)
driver.find_element_by_xpath('//*[@id="search-input-field"]').send_keys('London')
time.sleep(4)

In [46]:
# do click on search button
driver.find_element_by_xpath('//*[@id="predicted-search-results"]/li[2]').click()
driver.find_element_by_id('search-button').click()

In [53]:
#create empty list

hostel_name = []
distance = []
pvt_prices = []
dorms_price = []
rating = []
reviews = []
over_all = []
facilities = []
description =[]
product_url = []
# fetch the data
    
#Scraping hostel name
try:
    name = driver.find_elements_by_xpath("//h2[@class='title title-6']")
    for i in name:
        hostel_name.append(i.text)
except NoSuchElementException:
    hostel_name.append('-')
 #Scraping distance from city centre
try:
    dist = driver.find_elements_by_xpath("//div[@class='subtitle body-3']//a//span[1]")
    for i in dist:
        distance.append(i.text.replace('Hostel - ',''))
except NoSuchElementException:
    distance.append('-')
for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
    try:
        pvt_price = driver.find_element_by_xpath("//a[@class='prices']//div[1]//div")
        pvt_prices.append(pvt_price.text)
    except NoSuchElementException:
        pvt_prices.append('-')
#Scraping dorms from price
for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
    try:
        dorms = driver.find_element_by_xpath("//a[@class='prices']//div[2]//div")
        dorms_price.append(dorms.text)
    except NoSuchElementException:
        dorms_price.append('-')
 #Scraping facilities
try:
    fac1 = driver.find_elements_by_xpath("//div[@class='has-wifi']")
    fac2 = driver.find_elements_by_xpath("//div[@class='has-sanitation']")
    for i in fac1:
        for j in fac2:
            facilities.append(i.text +', '+ j.text )
except NoSuchElementException:
    facilities.append('-')
#Fetching url of each hostel
p_url = driver.find_elements_by_xpath("//div[@class='prices-col']//a[2]")
for i in p_url:
    product_url.append(i.get_attribute('href'))
    


In [57]:


for i in product_url:
    driver.get(i)
    time.sleep(5)
    try:
        rat = driver.find_element_by_xpath("//div[@class='score orange big' or @class='score gray big']")
        rating.append(rat.text)
    except NoSuchElementException:
        rating.append('-')
        
    #fetching total reviews
        
    try:
        rws = driver.find_element_by_xpath("//div[@class='reviews']")
        reviews.append(rws.text.replace('Total Reviews',''))
    except NoSuchElementException:
        reviews.append('-')
        #fetch overall review
    try:
        overall_rw = driver.find_element_by_xpath("//div[@class='keyword']//span")
        over_all.append(overall_rw.text)
    except NoSuchElementException:
        over_all.append('-')
    #fetch property description 
    try:
        disc = driver.find_element_by_xpath("//div[@class='content']")
        description.append(disc.text)
    except NoSuchElementException:
        over_all.append('-')
    
        
    

In [58]:
len(hostel_name),len(distance),len(pvt_prices),len(dorms_price),len(rating),len(reviews),len(over_all),len(facilities),len(description),len(product_url)


(17, 17, 17, 17, 17, 17, 17, 132, 17, 17)

In [59]:
facilities=facilities[:17]

In [61]:
Hostel = pd.DataFrame({})
Hostel['Hostel_Name'] = hostel_name
Hostel['Distance fron city centre'] = distance
Hostel['Ratings'] = rating
Hostel['Total_reviews'] = reviews
Hostel['Overall Reviews'] = over_all
Hostel['Privates from price'] = pvt_prices
Hostel['Dorms from price'] = dorms_price
Hostel['Description'] = description
Hostel

,Hostel_Name,Distance fron city centre,Ratings,Total_reviews,Overall Reviews,Privates from price,Dorms from price,Description
0,St Christopher's Village,1.8km from city centre,8.0,11328,Fabulous,Rs44444,Rs6698.61 Rs6029,"165 Borough High Street, London, England"
1,Selina Camden,5.5km from city centre,8.6,32,Fabulous,Rs44444,Rs6698.61 Rs6029,"89 Chalk Farm Rd, Chalk Farm, London, England"
2,Safestay London Elephant & Castle,1.7km from city centre,7.0,4369,Very Good,Rs44444,Rs6698.61 Rs6029,"144-152 Walworth Road, London, England"
3,Safestay London Kensington Holland Park,5.9km from city centre,6.5,1253,Good,Rs44444,Rs6698.61 Rs6029,"Holland Park Ave, Holland Walk, London, England"
4,Queen Elizabeth Chelsea,5.7km from city centre,6.6,3290,Good,Rs44444,Rs6698.61 Rs6029,"58 Bagleys Lane, London, England"
5,St Christopher's Hammersmith,7.5km from city centre,7.9,4052,Very Good,Rs44444,Rs6698.61 Rs6029,"28, Hammersmith Broadway, Hammersmith, London,..."
6,London Waterloo Hostel,0.7km from city centre,6.5,2506,Good,Rs44444,Rs6698.61 Rs6029,"73 Lambeth Walk, London, London, England"
7,St Christopher's Greenwich,7.6km from city centre,6.7,3215,Good,Rs44444,Rs6698.61 Rs6029,"189 Greenwich High Road, Greenwich, London, En..."
8,St Christopher's Shepherds Bush,7km from city centre,7.1,683,Very Good,Rs44444,Rs6698.61 Rs6029,"Belushis Bar, 13-15 Shepherds Bush Green., Lon..."
9,Saint James Backpackers,5.5km from city centre,6.7,1809,Good,Rs44444,Rs6698.61 Rs6029,"21 Longridge Road, London, England"
